<a href="https://colab.research.google.com/github/Rashi2011/Rashi-Madhukar/blob/master/Multiple%20Level%20Perceptron/n_layered_MLP_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#import Libraries
import numpy as np
import pandas as pd


In [3]:
#import Dataset
import io
from google.colab import files
uploaded=files.upload()
dataset = pd.read_csv(io.BytesIO(uploaded["Social_Network_Ads.csv"]))

x = dataset.iloc[:,1:4].values
y = dataset.iloc[:,4].values


Saving Social_Network_Ads.csv to Social_Network_Ads.csv


In [36]:
#converting column sex to numbers
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
x[:,0]=lb.fit_transform(x[:,0])

#Standard Scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x=sc.fit_transform(x)


print(x.shape,y.shape)

(3, 400) (400,)


In [37]:
#Initialize Parameters
def initialize_param(layers_dims):
  np.random.seed(105)
  parameters = {}                #empty dictionary
  L = len(layers_dims)           # 4-1 no of layers in the network
  for l in range(1,L):   #1,2,3
    parameters["W" + str(l)]= np.round(np.random.randn(layers_dims[l],layers_dims[l-1]),2)
    parameters["b"+ str(l)]= np.round(np.zeros((layers_dims[l],1)),2)

  return parameters

In [174]:

layers_dims = [3,5,3,1]
parameters = initialize_param(layers_dims)
#print(parameters)             #Dictionary
print("W1 = "+ str(parameters["W1"]))
print(parameters["W1"].shape)
print("b1 = "+ str(parameters["b1"]))
print("W2 = "+ str(parameters["W2"]))
print("b2 = "+ str(parameters["b2"]))
print("W3 = "+ str(parameters["W3"]))
print("b3 = "+ str(parameters["b3"]))



W1 = [[-0.25 -0.61 -0.73]
 [ 0.06  1.01 -0.13]
 [-1.53  0.47 -0.56]
 [-0.2   1.7   2.57]
 [ 0.56  0.95 -1.41]]
(5, 3)
b1 = [[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
W2 = [[ 1.42 -1.29  0.14  0.69 -0.3 ]
 [-1.58  0.29  0.89 -1.73 -0.09]
 [ 1.45  1.84  1.55  0.62  0.66]]
b2 = [[0.]
 [0.]
 [0.]]
W3 = [[ 0.14  2.09 -1.29]]
b3 = [[0.]]


In [175]:
#Activation Function
def sigmoid(z):
  A = 1/(1+np.exp(-z))
  return A

def relu(z):
  A = np.maximum(0,z)
  return A

def sign_f(AL,mean):
    if AL <= mean:
      return 0
    else:
      return 1

In [298]:
#forward
def forward(A, W, b):
  
  Z = np.dot(W,A) +b
  #print("Z.shape = ",Z)       
  
  return Z

In [299]:
#Activations forward
def activation_forward(A_prev, W, b ,activation):
  if activation == "sigmoid":
    #print("W,b",W.shape,b.shape)
    Z = forward(A_prev,W,b)
    A  = sigmoid(Z)
    

  elif activation == "tanh":
    #print("W,b",W.shape,b.shape)
    Z = forward(A_prev,W,b)
    A  =  np.tanh(Z)
    #print("Ashapes",A)
          
  return A,Z


In [300]:
#Forward Propagation
def forward_model(X,parameters):    
  cache = {}  #empty dictionary    
  A = X
  L = len(parameters)//2     #3
 
  for l in range(1,L):  #1,2
    A_prev =A
    A,cache["Z"+str(l)] = activation_forward(A_prev, parameters['W'+ str(l)], parameters['b'+ str(l)], activation = "tanh")  #i = 1,i = 2
    cache["A"+str(l)] = A
    
  #l=3
  l=L
  AL,cache["Z"+str(l)] = activation_forward(cache["A"+str(l-1)], parameters["W"+str(l)], parameters["b"+str(l)], activation = "sigmoid")  #i = 3
  cache["A"+str(l)] = AL
  
  return AL,cache

In [301]:
#Cost Function
def compute_cost(y_pred,Y):
  
  m = len(Y)
  #print(m)
  cost = -(1/m)*np.sum((Y*np.log(y_pred) + (1-Y)*np.log(1-y_pred)))
  #logprobs = np.multiply(np.log(AL),Y)
  #cost = np.sum(logprobs,axis = 1,keepdims = True)
  
  return cost

In [317]:

def sigmoid_backward(dA,AL):
  
  dZ = np.multiply(AL,(1-AL))
  Loss = np.multiply(dZ,AL)             #(1x207)
  return Loss


def tanh_backward(dA,AL):
  dZ = 1-np.power(dA,2)

  #print(dZ.shape)               #(5x207)
  dZ = np.multiply(dZ,AL)
  Loss = np.multiply(dA,dZ)
  return Loss

def relu_backward(dA,AL):
  dZ= np.zeros((AL.shape))
  for i in range(AL.shape[0]):
    for j in range(AL.shape[1]):
        
        if AL[i][j] <= 0:
          dZ[i][j] = dZ[i][j]+ 0
        else:
          dZ[i][j] =dZ[i][j] + 1
  #print(dZ.shape)               #(5x207)
  dZ = np.multiply(dZ,AL)
  Loss = np.multiply(dA,dZ)

  return Loss

In [318]:
#Backward 
def backward(dA,W,A_prev,Loss): 
  m = dA.shape[1]            
  dW = (1/m)*np.dot(Loss,A_prev.T)   #(1x3)
  dA_ = np.dot(W.T,dA)       #3x207
  #print("dA_",dA_.shape)
  db = (1/m)*np.sum(Loss,axis = 1,keepdims = True) 

  return dA_,dW,db

In [319]:
#Activation backward
def activation_backward(dA,W,A_prev,AL ,activation):
  
  if activation == "tanh":
    Loss = tanh_backward(dA, AL )
    dA_,dW ,db = backward(dA,W,A_prev,Loss)
  elif activation == "sigmoid":
    Loss = sigmoid_backward(dA,AL)
    dA_,dW ,db = backward(dA,W,A_prev,Loss)

  return dA_,dW,db

In [320]:
#backward Model
def backward_model(AL,X,Y, cache,parameters):
  
  grads = {}           #empty dictionary
  L = int(len(parameters)/2)              #3    #no of layers  
  #print(L)  #no of layers
  m = len(Y)                #no of training examples
  Y = Y.reshape(AL.shape)       #to ensure shape of Y = AL
  cache["A" + str(0)] = X
  #intialization of backward propagation
  dAL = AL- Y           #dA3
  #print("dAL",dAL.shape)
  #Sigmoid (for layer L = L-1 )       #L = 3
  
  grads["dA"+ str(L-1)],grads["dW"+ str(L)] ,grads["db"+ str(L)]  = activation_backward(dAL,parameters["W"+str(L)],cache["A" + str(L-1)],cache["A"+ str(L)] ,activation = "sigmoid")

  # for layer (L =L-2 to  L=0 ) 
  for l in reversed(range(L-1)):    #L-1 = 2, #(0 to 2) # means l = 0,1 # reversing it  l = 1,0
    if l!=0:
      grads["dA"+ str(l)],grads["dW"+ str(l+1)] ,grads["db"+ str(l+1)]  = activation_backward(grads["dA"+ str(l+1)], parameters["W"+ str(l+1)],cache["A"+str(l)],cache["A"+str(l+1)] ,activation = "tanh")

    else:
      _,grads["dW"+ str(l+1)] ,grads["db"+ str(l+1)]  = activation_backward(grads["dA"+ str(l+1)], parameters["W"+ str(l+1)],cache["A"+ str(l)] ,cache["A"+str(l+1)],activation = "tanh")
  return grads

In [321]:
#Update Parameters
def update_param(parameters,grads,learning_rate):
  L = len(parameters)//2   #3
  for l in range(L):  #0,1,2
    parameters["W"+str(l+1)] = parameters["W"+str(l+1)] - (learning_rate*(grads["dW"+ str(l+1)]))
    parameters["b"+str(l+1)] = parameters["b"+str(l+1)] - (learning_rate*(grads["db"+ str(l+1)]))

  return parameters

In [343]:
from sklearn.metrics import f1_score
#NN Model
def L_layer_model(X, Y, layers_dims, learning_rate , num_iterations):
  costs = []                #empty list
  acc = []
  parameters = initialize_param(layers_dims)
  L=len(parameters)//2
  for i in range(0,num_iterations):
    #Forward Propagation(relu to sigmoid)
    
    AL,cache = forward_model(X,parameters)
    
    #compute cost
    cost = compute_cost(AL,Y)
    
    #Backward Propagation
    grads = backward_model(AL,X,Y, cache,parameters)
    
    #Update Parameters
    parameters = update_param(parameters,grads,learning_rate)
    

    #Print the cost on every 100th training examples
    
    print ("Cost after iteration %i: %f" %(i, cost))
    
    #Converting to y_pred
    
    y_pred = []
    #print(AL.shape,Y.shape)

    
    for j in range(AL.shape[1]):
      y_pred.append(sign_f(AL[0][j],np.mean(AL)))
    #y_pred = np.asmatrix(y_pred)
    #print(y_pred.shape)
    #print(y_pred)
  
    costs.append(cost)
    #Accuracy
  
    print("Accuracy = {:.2f}".format(f1_score(Y,y_pred)))
    acc.append(f1_score(Y, y_pred,average = 'micro'))
         
  
  return parameters,costs,acc


In [344]:
layers_dims = [3,10,1]
parameters,cost,acc = L_layer_model(x, y, layers_dims, 0.01,num_iterations = 500)

Cost after iteration 0: 0.746627
Accuracy = 0.44
Cost after iteration 1: 0.747424
Accuracy = 0.44
Cost after iteration 2: 0.748227
Accuracy = 0.45
Cost after iteration 3: 0.749034
Accuracy = 0.45
Cost after iteration 4: 0.749846
Accuracy = 0.45
Cost after iteration 5: 0.750663
Accuracy = 0.45
Cost after iteration 6: 0.751484
Accuracy = 0.44
Cost after iteration 7: 0.752308
Accuracy = 0.44
Cost after iteration 8: 0.753136
Accuracy = 0.44
Cost after iteration 9: 0.753967
Accuracy = 0.44
Cost after iteration 10: 0.754800
Accuracy = 0.44
Cost after iteration 11: 0.755636
Accuracy = 0.44
Cost after iteration 12: 0.756474
Accuracy = 0.44
Cost after iteration 13: 0.757313
Accuracy = 0.45
Cost after iteration 14: 0.758154
Accuracy = 0.45
Cost after iteration 15: 0.758995
Accuracy = 0.45
Cost after iteration 16: 0.759837
Accuracy = 0.45
Cost after iteration 17: 0.760679
Accuracy = 0.45
Cost after iteration 18: 0.761521
Accuracy = 0.45
Cost after iteration 19: 0.762362
Accuracy = 0.45
Cost after